# Download and store data

This notebook contains information on downloading the Quandl Wiki stock prices and a few other sources that we use throughout the book. 

## Imports & Settings

In [13]:
import warnings
warnings.filterwarnings('ignore')

In [14]:
from pathlib import Path
import requests
from io import BytesIO
from zipfile import ZipFile, BadZipFile

import numpy as np
import pandas as pd
import pandas_datareader.data as web
from sklearn.datasets import fetch_openml

pd.set_option('display.expand_frame_repr', False)

## Set Data Store path

Modify path if you would like to store the data elsewhere and change the notebooks accordingly

In [15]:
DATA_STORE = Path('assets.h5')

## Quandl Wiki Prices

> Quandl has been [acuqired by NASDAQ](https://www.nasdaq.com/about/press-center/nasdaq-acquires-quandl-advance-use-alternative-data) in late 2018. In 2021, NASDAQ [integrated Quandl's data platform](https://data.nasdaq.com/). Free US equity data is still available under a [new URL](https://data.nasdaq.com/databases/WIKIP/documentation), subject to the limitations mentioned below.

[NASDAQ](https://data.nasdaq.com/) makes available a [dataset](/home/stefan/drive/machine-learning-for-trading/data/create_datasets.ipynb) with stock prices, dividends and splits for 3000 US publicly-traded companies. Prior to its acquisition (April 11, 2018), Quandl announced the end of community support (updates). The historical data are useful as a first step towards demonstrating the application of the machine learning solutions, just ensure you design and test your own algorithms using current, professional-grade data.

1. Follow the instructions to create a free [NASDAQ account](https://data.nasdaq.com/sign-up)
2. [Download](https://data.nasdaq.com/tables/WIKIP/WIKI-PRICES/export) the entire WIKI/PRICES data
3. Extract the .zip file,
4. Move to this directory and rename to wiki_prices.csv
5. Run the below code to store in fast HDF format (see [Chapter 02 on Market & Fundamental Data](../02_market_and_fundamental_data) for details).

In [18]:
df = (pd.read_csv('wiki_prices.csv',
                 parse_dates=['date'],
                 index_col=['date', 'ticker'],
                 infer_datetime_format=True)
     .sort_index())

# Update the info() method call
print(df.info(show_counts=True))

# Storing the dataframe into the HDF5 file
with pd.HDFStore(DATA_STORE) as store:
    store.put('quandl/wiki/prices', df)


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 15389314 entries, (Timestamp('1962-01-02 00:00:00'), 'ARNC') to (Timestamp('2018-03-27 00:00:00'), 'ZUMZ')
Data columns (total 12 columns):
 #   Column       Non-Null Count     Dtype  
---  ------       --------------     -----  
 0   open         15388776 non-null  float64
 1   high         15389259 non-null  float64
 2   low          15389259 non-null  float64
 3   close        15389313 non-null  float64
 4   volume       15389314 non-null  float64
 5   ex-dividend  15389314 non-null  float64
 6   split_ratio  15389313 non-null  float64
 7   adj_open     15388776 non-null  float64
 8   adj_high     15389259 non-null  float64
 9   adj_low      15389259 non-null  float64
 10  adj_close    15389313 non-null  float64
 11  adj_volume   15389314 non-null  float64
dtypes: float64(12)
memory usage: 1.4+ GB
None


### Wiki Prices Metadata

> QUANDL used to make some stock meta data be available on its website; I'm making the file available to allow readers to run some examples in the book:

Instead of using the QUANDL API, load the file `wiki_stocks.csv` as described and store in HDF5 format.

In [20]:
df = pd.read_csv('wiki_stocks.csv')

# Output dataframe information with updated 'show_counts' argument
print(df.info(show_counts=True))

# Storing the dataframe into the HDF5 file
with pd.HDFStore(DATA_STORE) as store:
    store.put('quandl/wiki/stocks', df)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3199 entries, 0 to 3198
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   code    3199 non-null   object
 1   name    3199 non-null   object
dtypes: object(2)
memory usage: 50.1+ KB
None


## S&P 500 Prices

The following code downloads historical S&P 500 prices from FRED (only last 10 years of daily data is freely available)

In [21]:
df = web.DataReader(name='SP500', data_source='fred', start=2009).squeeze().to_frame('close')
print(df.info())
with pd.HDFStore(DATA_STORE) as store:
    store.put('sp500/fred', df)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2610 entries, 2014-09-22 to 2024-09-20
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   close   2517 non-null   float64
dtypes: float64(1)
memory usage: 40.8 KB
None


Alternatively, download S&P500 data from [stooq.com](https://stooq.com/q/?s=%5Espx&c=1d&t=l&a=lg&b=0); at the time of writing the data was available since 1789. You can switch from Polish to English on the lower right-hand side.

We store the data from 1950-2020:

In [22]:
sp500_stooq = (pd.read_csv('^spx_d.csv', index_col=0,
                     parse_dates=True).loc['1950':'2019'].rename(columns=str.lower))
print(sp500_stooq.info())

FileNotFoundError: [Errno 2] No such file or directory: '^spx_d.csv'

In [10]:
with pd.HDFStore(DATA_STORE) as store:
    store.put('sp500/stooq', sp500_stooq)

### S&P 500 Constituents

The following code downloads the current S&P 500 constituents from [Wikipedia](https://en.wikipedia.org/wiki/List_of_S%26P_500_companies).

In [24]:
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
df = pd.read_html(url, header=0)[0]

In [25]:
df.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [27]:
# Check the current number of columns in the DataFrame
print(df.columns)

# Assuming there are actually 8 columns in the DataFrame
# Modify the column list to match the number of columns in the DataFrame
df.columns = ['ticker', 'name', 'gics_sector', 'gics_sub_industry',
              'location', 'first_added', 'cik', 'founded']

# Drop the 'sec_filings' column only if it exists
if 'sec_filings' in df.columns:
    df = df.drop('sec_filings', axis=1)

# Set 'ticker' as the index
df = df.set_index('ticker')

print(df.head())


Index(['Symbol', 'Security', 'GICS Sector', 'GICS Sub-Industry',
       'Headquarters Location', 'Date added', 'CIK', 'Founded'],
      dtype='object')
                       name             gics_sector               gics_sub_industry                 location first_added      cik      founded
ticker                                                                                                                                        
MMM                      3M             Industrials        Industrial Conglomerates    Saint Paul, Minnesota  1957-03-04    66740         1902
AOS             A. O. Smith             Industrials               Building Products     Milwaukee, Wisconsin  2017-07-26    91142         1916
ABT     Abbott Laboratories             Health Care           Health Care Equipment  North Chicago, Illinois  1957-03-04     1800         1888
ABBV                 AbbVie             Health Care                   Biotechnology  North Chicago, Illinois  2012-12-31  1551152  20

In [28]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 503 entries, MMM to ZTS
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   name               503 non-null    object
 1   gics_sector        503 non-null    object
 2   gics_sub_industry  503 non-null    object
 3   location           503 non-null    object
 4   first_added        503 non-null    object
 5   cik                503 non-null    int64 
 6   founded            503 non-null    object
dtypes: int64(1), object(6)
memory usage: 31.4+ KB
None


In [29]:
with pd.HDFStore(DATA_STORE) as store:
    store.put('sp500/stocks', df)

## Metadata on US-traded companies

The following downloads several attributes for [companies](https://www.nasdaq.com/screening/companies-by-name.aspx) traded on NASDAQ, AMEX and NYSE

> Update: unfortunately, NASDAQ has disabled automatic downloads. However, you can still access and manually download the files at the below URL when you fill in the exchange names. So for AMEX, URL becomes `https://www.nasdaq.com/market-activity/stocks/screener?exchange=AMEX&letter=0&render=download`.
>

In [33]:
import yfinance as yf

# Example: Fetch stock data for a few symbols from NASDAQ, AMEX, and NYSE
tickers = "AAPL MSFT TSLA"
stock_data = yf.download(tickers=tickers, period="1d")
print(stock_data)


[*********************100%***********************]  3 of 3 completed

Price                       Adj Close                           Close                            High                                Low                                Open                             Volume                    
Ticker                           AAPL        MSFT    TSLA        AAPL        MSFT    TSLA        AAPL       MSFT        TSLA        AAPL        MSFT        TSLA        AAPL        MSFT        TSLA       AAPL      MSFT      TSLA
Date                                                                                                                                                                                                                               
2024-09-20 00:00:00+00:00  228.199997  435.269989  238.25  228.199997  435.269989  238.25  233.089996  439.23999  243.990005  227.619995  434.220001  235.919998  229.970001  437.220001  241.520004  318582200  55155300  99760300


In [34]:
df.head()

,name,gics_sector,gics_sub_industry,location,first_added,cik,founded
ticker,,,,,,,
MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


### Convert market cap information to numerical format

Market cap is provided as strings so we need to convert it to numerical format.

In [41]:
import yfinance as yf
import pandas as pd

# List of tickers
tickers = ['AAPL', 'MSFT', 'GOOGL']  # Add more tickers if needed

# Initialize an empty list to store the data
data = []

# Loop through each ticker and retrieve data
for ticker in tickers:
    stock = yf.Ticker(ticker)
    hist = stock.history(period="1mo")  # Adjust period as needed, e.g., "1d", "1y", etc.
    
    # Get Market Cap from the stock info
    market_cap = stock.info.get('marketCap', 'N/A')  # Fetch market cap if available

    # Add a market cap column to the historical data
    hist['Market Cap'] = market_cap

    # Add ticker information
    hist['Ticker'] = ticker
    
    # Append to the data list
    data.append(hist)

# Concatenate all stock data into a single DataFrame
df = pd.concat(data)

# Reset index to have the dates and tickers as columns
df.reset_index(inplace=True)

# Show the DataFrame with Market Cap
print(df.head())


                       Date        Open        High         Low       Close    Volume  Dividends  Stock Splits     Market Cap Ticker
0 2024-08-21 00:00:00-04:00  226.520004  227.979996  225.050003  226.399994  34765500        0.0           0.0  3469575716864   AAPL
1 2024-08-22 00:00:00-04:00  227.789993  228.339996  223.899994  224.529999  43695300        0.0           0.0  3469575716864   AAPL
2 2024-08-23 00:00:00-04:00  225.660004  228.220001  224.330002  226.839996  38677300        0.0           0.0  3469575716864   AAPL
3 2024-08-26 00:00:00-04:00  226.759995  227.279999  223.889999  227.179993  30602200        0.0           0.0  3469575716864   AAPL
4 2024-08-27 00:00:00-04:00  226.000000  228.850006  224.889999  228.029999  35934600        0.0           0.0  3469575716864   AAPL


In [42]:
import yfinance as yf
import pandas as pd

# List of tickers
tickers = ['AAPL', 'MSFT', 'GOOGL']  # Add more tickers if needed

# Initialize an empty list to store the data
data = []

# Loop through each ticker and retrieve data
for ticker in tickers:
    stock = yf.Ticker(ticker)
    
    # Get Market Cap from the stock info
    market_cap = stock.info.get('marketCap', 0)  # Fetch market cap if available, else 0

    # Append data as dictionary
    data.append({'ticker': ticker, 'marketcap': market_cap})

# Create a DataFrame
mcap = pd.DataFrame(data)

# Convert to a readable format (e.g., M for millions, B for billions)
# We'll categorize values into 'M' or 'B' based on their size
mcap['suffix'] = pd.cut(mcap['marketcap'],
                        bins=[0, 1e9, float('inf')],
                        labels=['M', 'B'])

# Filter for only values with 'M' (millions) or 'B' (billions)
mcap = mcap[mcap['suffix'].isin(['M', 'B'])]

# Convert market cap to a consistent numeric format (in millions or billions)
mcaps = {'M': 1e6, 'B': 1e9}
for symbol, factor in mcaps.items():
    mcap.loc[mcap['suffix'] == symbol, 'marketcap'] /= factor  # Normalize the values

# Show the resulting DataFrame
mcap.info()
print(mcap)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   ticker     3 non-null      object  
 1   marketcap  3 non-null      float64 
 2   suffix     3 non-null      category
dtypes: category(1), float64(1), object(1)
memory usage: 303.0+ bytes
  ticker    marketcap suffix
0   AAPL  3469.575717      B
1   MSFT  3235.379151      B
2  GOOGL  2019.943776      B


Keep only values with value units:

In [44]:
df['marketcap'] = mcap.marketcap
df.marketcap.describe(percentiles=np.arange(.1, 1, .1).round(1)).apply(lambda x: f'{int(x):,d}')

count        3
mean     2,908
std        778
min      2,019
10%      2,263
20%      2,506
30%      2,749
40%      2,992
50%      3,235
60%      3,282
70%      3,329
80%      3,375
90%      3,422
max      3,469
Name: marketcap, dtype: object

### Store result

The file `us_equities_meta_data.csv` contains a version of the data used for many of the examples. Load using 
```
df = pd.read_csv('us_equities_meta_data.csv')
```
and proceed to store in HDF5 format.

In [45]:
df = pd.read_csv('us_equities_meta_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6834 entries, 0 to 6833
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ticker     6834 non-null   object 
 1   name       6834 non-null   object 
 2   lastsale   6718 non-null   float64
 3   marketcap  5766 non-null   float64
 4   ipoyear    3038 non-null   float64
 5   sector     5288 non-null   object 
 6   industry   5288 non-null   object 
dtypes: float64(3), object(4)
memory usage: 373.9+ KB


In [46]:
with pd.HDFStore(DATA_STORE) as store:
    store.put('us_equities/stocks', df.set_index('ticker'))

## MNIST Data

In [47]:
mnist = fetch_openml('mnist_784', version=1)

In [48]:
print(mnist.DESCR)

**Author**: Yann LeCun, Corinna Cortes, Christopher J.C. Burges  
**Source**: [MNIST Website](http://yann.lecun.com/exdb/mnist/) - Date unknown  
**Please cite**:  

The MNIST database of handwritten digits with 784 features, raw data available at: http://yann.lecun.com/exdb/mnist/. It can be split in a training set of the first 60,000 examples, and a test set of 10,000 examples  

It is a subset of a larger set available from NIST. The digits have been size-normalized and centered in a fixed-size image. It is a good database for people who want to try learning techniques and pattern recognition methods on real-world data while spending minimal efforts on preprocessing and formatting. The original black and white (bilevel) images from NIST were size normalized to fit in a 20x20 pixel box while preserving their aspect ratio. The resulting images contain grey levels as a result of the anti-aliasing technique used by the normalization algorithm. the images were centered in a 28x28 image b

In [49]:
mnist.keys()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [50]:
mnist_path = Path('mnist')
if not mnist_path.exists():
    mnist_path.mkdir()

In [51]:
np.save(mnist_path / 'data', mnist.data.astype(np.uint8))
np.save(mnist_path / 'labels', mnist.target.astype(np.uint8))

## Fashion MNIST Image Data

We will use the Fashion MNIST image data created by [Zalando Research](https://github.com/zalandoresearch/fashion-mnist) for some demonstrations.

In [52]:
fashion_mnist = fetch_openml(name='Fashion-MNIST')

In [53]:
print(fashion_mnist.DESCR)

**Author**: Han Xiao, Kashif Rasul, Roland Vollgraf  
**Source**: [Zalando Research](https://github.com/zalandoresearch/fashion-mnist)  
**Please cite**: Han Xiao and Kashif Rasul and Roland Vollgraf, Fashion-MNIST: a Novel Image Dataset for Benchmarking Machine Learning Algorithms, arXiv, cs.LG/1708.07747  

Fashion-MNIST is a dataset of Zalando's article images, consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes. Fashion-MNIST is intended to serve as a direct drop-in replacement for the original MNIST dataset for benchmarking machine learning algorithms. It shares the same image size and structure of training and testing splits. 

Raw data available at: https://github.com/zalandoresearch/fashion-mnist

### Target classes
Each training and test example is assigned to one of the following labels:
Label  Description  
0  T-shirt/top  
1  Trouser  
2  Pullover  
3  Dress  
4  

In [54]:
label_dict = {0: 'T-shirt/top',
              1: 'Trouser',
              2: 'Pullover',
              3: 'Dress',
              4: 'Coat',
              5: 'Sandal',
              6: 'Shirt',
              7: 'Sneaker',
              8: 'Bag',
              9: 'Ankle boot'}

In [55]:
fashion_path = Path('fashion_mnist')
if not fashion_path.exists():
    fashion_path.mkdir()

In [56]:
pd.Series(label_dict).to_csv(fashion_path / 'label_dict.csv', index=False, header=None)

In [57]:
np.save(fashion_path / 'data', fashion_mnist.data.astype(np.uint8))
np.save(fashion_path / 'labels', fashion_mnist.target.astype(np.uint8))


## Bond Price Indexes

The following code downloads several bond indexes from the Federal Reserve Economic Data service ([FRED](https://fred.stlouisfed.org/))

> Warning: Unfortunately, most of this data has been [recently removed](https://news.research.stlouisfed.org/2022/01/ice-benchmark-administration-ltd-iba-data-to-be-removed-from-fred/) from the FRED service. It is not important for the examples in the book, so you can just ignore this.

In [59]:
import pandas_datareader.data as web
import pandas as pd

# Dictionary of securities and their FRED codes
securities = {
    'BAMLCC0A0CMTRIV'   : 'US Corp Master TRI',
    'BAMLHYH0A0HYM2TRIV': 'US High Yield TRI',
    'BAMLEMCBPITRIV'    : 'Emerging Markets Corporate Plus TRI',
    'GOLDAMGBD228NLBM'  : 'Gold (London, USD)',
    'DGS10'             : '10-Year Treasury CMR',
}

# Initialize an empty DataFrame
df_list = []

# Try to fetch each security separately to handle missing series gracefully
for symbol, name in securities.items():
    try:
        df_temp = web.DataReader(name=symbol, data_source='fred', start='2000-01-01')
        df_temp = df_temp.rename(columns={symbol: name})
        df_list.append(df_temp)
        print(f"Successfully retrieved data for {name}")
    except Exception as e:
        print(f"Failed to retrieve data for {name}: {e}")

# Combine the DataFrames
if df_list:
    df = pd.concat(df_list, axis=1)
    df = df.dropna(how='all').resample('B').mean()

    # Save to HDF5 store
    DATA_STORE = 'financial_data.h5'
    with pd.HDFStore(DATA_STORE) as store:
        store.put('fred/assets', df)
    print("Data saved to HDF5 store.")
else:
    print("No data retrieved.")


Successfully retrieved data for US Corp Master TRI
Successfully retrieved data for US High Yield TRI
Successfully retrieved data for Emerging Markets Corporate Plus TRI
Failed to retrieve data for Gold (London, USD): Unable to read URL: https://fred.stlouisfed.org/graph/fredgraph.csv?id=GOLDAMGBD228NLBM
Response Text:
b'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n    <meta charset="utf-8">\r\n    <meta http-equiv="X-UA-Compatible" content="IE=edge">\r\n    <meta name="viewport" content="width=device-width, initial-scale=1">\r\n    <title>Error - St. Louis Fed</title>\r\n    <meta name="description" content="">\r\n    <meta name="keywords" content="">    \r\n    <link rel="stylesheet" type="text/css" href="/assets/bootstrap/dist/css/bootstrap.min.css">\r\n    <link rel="stylesheet" type="text/css" href="/css/home.min.css?1553087253">\r\n    <link rel="stylesheet" type="text/css" href="/assets/fontawesome-free/css/all.min.css">\r\n    <link rel="stylesheet" type="text/css" href="/as